### Keşifçi Veri Analizi ( EDA )

In [15]:
################################################
# KNN
################################################

# 1. Exploratory Data Analysis
# 2. Data Preprocessing & Feature Engineering
# 3. Modeling & Prediction
# 4. Model Evaluation
# 5. Hyperparameter Optimization
# 6. Final Model

import pandas as pd
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)

In [16]:
################################################
# 1. Exploratory Data Analysis
################################################

df = pd.read_csv("datasets/diabetes.csv")

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [17]:
df.shape

(768, 9)

In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [19]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

### Veri Ön İşleme ( Data Pre-Processing )

* Bu problemin özelinde bir standartlaştırma işlemi yapmamız gerekiyor. Kullanacak olduğumuz knn yöntemi uzaklık temelli bir yöntemdir. Uzaklık temelli yöntemlerde ve gradient descent temelli yöntemlerde değişkenlerin standart olması elde edilecek sonuçların ya daha hızlı ya da daha doğru olmasını diğer ifadesiyle daha başarılı olunmasını sağlayacaktır. Bu sebeple elimizdeki bağımsız değişkenleri standartlaştırma işlemine sokuyor olacağız. 

In [20]:
################################################
# 2. Data Preprocessing & Feature Engineering
################################################

y = df["Outcome"]
X = df.drop(["Outcome"], axis=1)

X_scaled = StandardScaler().fit_transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [21]:
# Bağımlı değişkenimiz:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [22]:
# Bağımsız değişkenlerimiz:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


In [23]:
X_scaled
# Bir numpy array'i dönüyor. Değişken değerleri standartlaştırılmış olsa da elimizdeki numpy array'İ istediğimiz bilgileri
# taşımıyor. Mesela sütun isimleri yok dolayısıyla bu sütun isimlerini ekleme ihtiyacımız vardır. Bunu da 
# X = pd.DataFrame(X_scaled, columns=X.columns) şeklinde yapıyoruz.

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [26]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496
...,...,...,...,...,...,...,...,...
763,1.827813,-0.622642,0.356432,1.722735,0.870031,0.115169,-0.908682,2.532136
764,-0.547919,0.034598,0.046245,0.405445,-0.692891,0.610154,-0.398282,-0.531023
765,0.342981,0.003301,0.149641,0.154533,0.279594,-0.735190,-0.685193,-0.275760
766,-0.844885,0.159787,-0.470732,-1.288212,-0.692891,-0.240205,-0.371101,1.170732


### Modelleme

In [30]:
################################################
# 3. Modeling & Prediction
################################################

# Modeli kurduk:
knn_model = KNeighborsClassifier().fit(X, y)

# Diyelim ki tahmin yapmak istiyoruz, Rastgele 1 hasta seçiyoruz:
random_user = X.sample(1, random_state=45)

# predict metodu hangi özellikleri knn modeline sormamı istersin, ben aracıyım senin bana verdiğin özelliklere göre gidicem ona
# elimde bu bu özellikler var söyle bakalım bu kişi diyabet mi değil mi diyeceğim der.
knn_model.predict(random_user)

array([1], dtype=int64)

### Model Başarı Değerlendirme

In [33]:
################################################
# 4. Model Evaluation
################################################

# knn modelini kullanarak bütün gözlem birimleri için tahmin yapıp bunları bir yerde saklamamız lazım. y_pred diyerek bunları 
# saklayalım. Bunu confussion matrix'i hesaplamak için kullanacağız. Bunun üzerinden accucay ve f1 gibi metrikleri hesaplıyor
# olacağız.
# Confusion matrix için y_pred:
y_pred = knn_model.predict(X)
y_pred

array([1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [34]:
# Bir de AUC'i hesaplamaya ihtiyacımız vardı. Bunu da olasık değerleri üzerinden yapabiliyorduk. ROC eğrisinin altında kalan 
# alanı ifade eden AUC'i hesaplamak istiyoruz. Bunu nasıl heaplıyorduk, ROC eğrisi üzerinden gidiyorduk, ROC eğrisini nasıl
# hesaplıyorduk, classification_thresholdları değiştirerek yani örneğin 0.3, 0.5, 0.7 gibi farklı aralıklara giderek bu farklı
# aralıklar üzerinden TP rate ve FP rate'lre göre ROC eğrisini oluşturuyorduk ve bu eğrinin altında kalan alanı AUC'i 
# hesaplıyorduk. Dolayısıyla bize bu şekilde tahmin edilen değerler (y_pred) değil de 1 sınıfına ait olma olasılıkları lazım
# AUC'i ve ROC'u hesaplamak için.

# AUC için y_prob:
y_prob = knn_model.predict_proba(X)[:, 1]
y_prob

array([0.8, 0. , 0.8, 0. , 0.4, 0.2, 0.2, 0.4, 0.6, 0.4, 0. , 0.6, 0.4,
       1. , 0.4, 0.4, 0.8, 0.6, 0.2, 0.4, 0.4, 0. , 0.8, 0.6, 0.6, 0.6,
       0.8, 0. , 0.4, 0.4, 0.6, 0.6, 0. , 0. , 0.8, 0.2, 0.4, 0.4, 0.2,
       0.6, 0.8, 0.6, 0.2, 1. , 0.6, 0.8, 0.4, 0. , 0.2, 0.2, 0. , 0. ,
       0. , 1. , 0.4, 0. , 0.8, 0. , 0.6, 0. , 0. , 0.8, 0. , 0.2, 1. ,
       0. , 0.4, 0.8, 0. , 0.6, 0.2, 0.6, 0.6, 0. , 0. , 0. , 0. , 0.2,
       0.8, 0. , 0. , 0. , 0.2, 0. , 0.6, 0.2, 0.2, 0. , 0.6, 0. , 0. ,
       0.4, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. , 0.6, 0.8, 0.2, 0. , 0. ,
       0.2, 0. , 0. , 0.4, 0.2, 0.2, 0.8, 0.6, 0. , 0.2, 0.2, 0.6, 0.6,
       0. , 0. , 0. , 0.8, 0.2, 0. , 0.2, 0.2, 0.2, 0.2, 0.2, 0.8, 0.4,
       0.8, 0.8, 0.8, 0.2, 0.2, 0.2, 0.2, 0. , 0.4, 0.2, 0.4, 0. , 0. ,
       0.6, 0.6, 0. , 0.2, 0.4, 0.2, 0. , 0.2, 0. , 0.6, 0.4, 0.8, 0.8,
       0.2, 0. , 0. , 0.8, 0.2, 0. , 0.2, 0. , 0.4, 0.2, 0. , 0.4, 0.4,
       0. , 0.6, 0.4, 0.2, 0.2, 0. , 0.8, 0.4, 0.8, 0.4, 0.4, 0.

In [35]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.90      0.87       500
           1       0.79      0.70      0.74       268

    accuracy                           0.83       768
   macro avg       0.82      0.80      0.81       768
weighted avg       0.83      0.83      0.83       768



* 1 sınıfı için precision değerini recall değerini f1 skorunu ve sınıf bilgilerini belirtmiş. Accuracy değerimiz 0.83'tür. Başrılı sınıflandırma oranı bire bir dediğimiz ve sıfıra sıfır dediğimiz durumlar bölü tüm durumlardı. Bu 100 kişiden 83'üne diabet ya da diabet değil dediğimizde bu doğru olmuş oluyor. %17'sinde tahminimiz bşarısız olmuş oluyor.

* Accuracy her zaman doğru bilgi vermeyebilir eğer dengesiz veri varsa başka metriklere de bakmak gerekir demiştik. 
* Precision 1 olarak tahmin ettiklerimizin başarısıydı. Yani 1 sınıfına yönelik tahminlerimizin başarısını ifade ediyordu.
* Recal gerçekte 1 olanları 1 olarak tahmin etme başarımızı ifade ediyordu.
* F1 skoru ise presicion ve recall'un harmonik ortalamasıydı.

* Çıkan değerlere baktığımda (harmonik ortalama 0.74) 0.70'in üzerindeyiz. Bu başarılı kabul edilebilir. Bu başarıyı arttırmamız da gerekiyor tabii ki. 
* macro avg = aritmetik ortalama
* weighted avg = ağırlıklı ortalama

In [36]:
# AUC
roc_auc_score(y, y_prob)

0.9017686567164179

* Tüm veri ile bir model kurduk ve kurmuş olduğumuz modelin başarısını yine modeli kurarken kullandığımız veri ile test ettik. Aslında olması gereken şey modeli görmediği veri ile test etmektir. Bunun için iki yol öğrenmiştik. Birincisi holdout yöntemi, ikincisi de cross-validation yöntemiydi. 

In [38]:
# Cross Validation
cv_results = cross_validate(knn_model, X, y, cv=5, scoring=["accuracy", "f1", "roc_auc"])

# cross_val_score isimli başka bir metod var. cross_validate metodunun cross_val_Score metodundan farkı birden fazla metriğe 
# göre de değerlendirme yapabiliyor olmasıdır. Yani veri setindeki başarımızı değerlendirdik ama bir doğrulama ihtiyacımız var 
# çünkü bütün veriyle modeli kurduk ve bütün veriyle test ettik. Şimdi çapraz doğrulama yöntemiyle doğrulama yapmak istiyoruz.
# Hatamızın geçerliliğini incelemek istiyoruz. Bunun için tek bir metriğe değil birden fazla metriğe bakacağız.

cv_results

{'fit_time': array([0.00498652, 0.00199461, 0.00199413, 0.00299191, 0.00199533]),
 'score_time': array([0.01396179, 0.01097107, 0.00997353, 0.00997353, 0.00997257]),
 'test_accuracy': array([0.72077922, 0.73376623, 0.71428571, 0.77124183, 0.7254902 ]),
 'test_f1': array([0.58252427, 0.60952381, 0.54166667, 0.63917526, 0.58      ]),
 'test_roc_auc': array([0.77555556, 0.78759259, 0.73194444, 0.83226415, 0.77528302])}

* İçerisinde fit süreleri, prediction süreleri (score_time), test'in accuracysi, test'in f1 ve roc auc değerlerini içinde tutuyor. 5 katlı çapraz doğrulama yaptığımız için veriyi 5'e böldük. 4 tanesiyle modeli kurduk, 1 tanesiyle de test ettik. 4'lü ve 1'li setler kendi aralaerında tekrar tekrar değişerek model kurulup test edildi. Böylece roc auc değerleri ortaya çıktı. Dolayısıyla biz bunların ortalamasını aldığımızda 5 katlı çapraz doğrulamanın bütün test skorlarının ortalamasını almış olacağız. Bu işlemleri de hemen gerçekleştirelim:

In [39]:
cv_results['test_accuracy'].mean()

# eski acc değerimiz 0.83'müş. 0.73'e düştü.

0.733112638994992

In [40]:
cv_results['test_f1'].mean()

# Sonuç ilgi çekici çünkü başarılı gibi gözüküyorduk ancak ortalama 0.59 geldi.

0.5905780011534191

In [41]:
cv_results['test_roc_auc'].mean()

# %90'dan %78'e düşmüş.

0.7805279524807827

* Modeli kurduğumuz veriyi modelin performansını değerlendirmek için kullandığımızda aslında ortaya bir miktar yanlılık çıktı. Bu yanlılık ne getirir ne götürür sonuçları doğru değerlendirmemizi engeller. Normalde burada dengesiz veri problemi var mı yok mu gibi soru işareti oluştuğunda f1 skorunu 0.74 olarak gördüğümüzde bu bizi bundan uzaklaştırabilir. Tamam çok ciddi olamsa da bir miktar dengesizlik var gibi acaba göz ardı edilmeli mi gibi bize bir yorum yaptırabilir.
* Dolayısıyla veri setini cross validation yöntemi ile bölerek ayrı parçalarla model kurup diğer parçalarla test ettiğimizde bütün veriyle model kurmaya göre daha farklı bir sonuç aldık ve bu sonuçlar daha güvenilirdir. 

##### DİKKAT : Bu Başarı Skorları Nasıl Arttırılabilir?

* Bu başarı sonuçları birkaç şekilde arttırılabilir. 
* 1 - Veri boyutu arttırılabilir. Gözlem sayısını arttırmayı kastediyoruz. Örnek boyutu da diyebiliriz.
* 2 - Veri ön işleme işlemleri detaylandırılabilir.
* 3 - Özellik mühendisliği, yeni değişkenler türetilebilir.
* 4 - İlgili algoritma için optimizasyonlar yapılabilir.

In [42]:
# knn yönteminin bir dışsal hiperparametresi vardır. Komşuluk sayısı hiperparametresidir. Bu komşuluk sayısı hiperparametresi 
# değiştirilebilirdir.

knn_model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

* Parametre modellerin veri içinden öğrendiği ağırlıklardır. Ağırlıklar o parametrelerin tahmincileriydi. Yani özetle parametre dediğimiz şey veri içerisinden öğrenilmektedir.
* Hiperparametre ise kullanıcı tarafından tanımlanması gereken, dışsal ve veri seti içerisinden öğrenilemeyen parametrelerdir. Bunalar hiperparametre denir. Fakat biz yine veriyi kullanarak kullanıcı olarak bir hiperparametre seti vereceğiz. Mesela 10, 20 tane komşuluk sayısı hiperparametresi vereceğiz. Bunları tek tek deneyeceğiz ve denemeler neticesinde hatamıza bakıp böylece normalde veriden öğrenilemeyen kullanıcı tarafından verilmesi gereken bu hiperparametreleri bir set olarak gönderip al bakalım 6'ya bak, 10'a bak, 20'ye bak, dene bunları ondan sonra en düşük hatayı veren hiperparametreyi ya da hiperparametre setini bize nihai olarak göster. Biz de ondan sonra final bir model kuralım. Bu hiperparametre değerinin ön tanımlı değerini değiştirelim, ona göre bir daha hatamıza bakalım diyerek algoritmanın hiperparametresini optimize ediyor olacağız.